# Aerosol calibration

Use the Smith et al. 2021 calibrations to 12 CMIP6 models, with a uniform distribution of 1750-2010 ERFaci of 0 to -3 W m-2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import json
import scipy.stats

from tqdm import tqdm

In [ ]:
with open('../data/smith2021aerosol/cmip6_aerosol_coefficients.json') as f:
    data = json.load(f)

In [ ]:
log_n0_samp = np.zeros(12)
log_n1_samp = np.zeros(12)

In [ ]:
for imodel, model in enumerate(data):
    if imodel==12:
        break
    log_n0_samp[imodel] = np.log(data[model]['ERFaci']['n0'])
    log_n1_samp[imodel] = np.log(data[model]['ERFaci']['n1'])

In [ ]:
kde = scipy.stats.gaussian_kde([log_n0_samp, log_n1_samp])
aci_sample=kde.resample(size=50000, seed=63648708)

In [ ]:
emissions = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_emissions = pd.concat([emissions.loc[(
        (emissions.Variable=='Emissions|BC')|
        (emissions.Variable=='Emissions|OC')|
        (emissions.Variable=='Emissions|Sulfur')|
        (emissions.Variable=='Emissions|NOx')|
        (emissions.Variable=='Emissions|NH3')
    ) & (emissions.Scenario=='ssp245') & (emissions.Region=='World'), 'Variable'], emissions.loc[(
        (emissions.Variable=='Emissions|BC')|
        (emissions.Variable=='Emissions|OC')|
        (emissions.Variable=='Emissions|Sulfur')|
        (emissions.Variable=='Emissions|NOx')|
        (emissions.Variable=='Emissions|NH3')
    ) & (emissions.Scenario=='ssp245') & (emissions.Region=='World'), '1750':'2100']], axis=1)#.interpolate(axis=1).T
df_emissions.set_index('Variable', inplace=True)
df_emissions = df_emissions.interpolate(axis=1).T
df_emissions.rename(
    columns={
        'Emissions|BC': 'BC',
        'Emissions|OC': 'OC',
        'Emissions|Sulfur': 'SO2',
        'Emissions|NOx': 'NOx',
        'Emissions|NH3': 'NH3',
    }, inplace=True
)
# only keep cols we want
emissions = df_emissions[['SO2', 'BC', 'OC', 'NH3', 'NOx']]
emissions.index = emissions.index.astype('int')
emissions.index.name='year'
emissions.columns.name=None

In [ ]:
emissions

In [ ]:
def ghan(x, beta, n0, n1):
    return -beta*np.log(1 + x[0]/n0 + x[1]/n1)

In [ ]:
samples = 50000

In [ ]:
erfaci_sample = scipy.stats.uniform.rvs(-3, 3, size=samples, random_state = 71271)

In [ ]:
erfaci = np.zeros((351,samples))
beta = np.zeros(samples)
for i in tqdm(range(samples)):
    ts2010 = np.mean(ghan([emissions.loc[2005:2015,'SO2'], emissions.loc[2005:2015,'BC']+emissions.loc[2005:2015,'OC']], 0.97, np.exp(aci_sample[0,i]), np.exp(aci_sample[1,i])))
    ts1850 = ghan([emissions.loc[1850,'SO2'],emissions.loc[1850,'BC']+emissions.loc[1850,'OC']], 0.97, np.exp(aci_sample[0,i]), np.exp(aci_sample[1,i])) 
    ts1750 = ghan([emissions.loc[1750,'SO2'],emissions.loc[1750,'BC']+emissions.loc[1750,'OC']], 0.97, np.exp(aci_sample[0,i]), np.exp(aci_sample[1,i]))
    erfaci[:,i] = (
        ghan([emissions['SO2'],emissions['BC']+emissions['OC']], 0.97, np.exp(aci_sample[0,i]), np.exp(aci_sample[1,i])) 
        - ts1750
    )/(ts2010-ts1850)*(erfaci_sample[i])
    beta[i] = erfaci_sample[i] / (ts2010-ts1750)

In [ ]:
pl.plot(erfaci[:, :]);
#pl.ylim(-4, 0.5);

In [ ]:
np.sum(np.isnan(erfaci))

In [ ]:
df = pd.DataFrame({
    'shape_so2' : np.exp(aci_sample[0,:]),
    'shape_bcoc' : np.exp(aci_sample[1,:]), 
    'beta' : beta
})

In [ ]:
df

In [ ]:
df.to_csv('../data/parameter_sets/erfaci.csv', index=False)